In [1]:
ls

data/   etl.py        README.md               spark-warehouse/
dl.cfg  __pycache__/  songs_table_image1.PNG  Untitled.ipynb


In [2]:
import configparser
from datetime import datetime
import os
import calendar
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType
from pyspark.sql.functions import udf, col, to_timestamp, monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Dat, TimestampType
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [3]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']


In [4]:
"""This function scans all the spark components"""
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [5]:
input_data =''

In [6]:
 """ Processing log data from the JSON file provided in the S3 bucket"""  
    # get filepath to log data file
log_data = input_data + 'data/*.json'

In [7]:
log_data_schema = StructType([
        StructField("artist", StringType(), True),
        StructField("auth", StringType(), True),
        StructField("firstName", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("itemInSession", LongType(), True),
        StructField("lastName", StringType(), True),
        StructField("length", DoubleType(), True),
        StructField("level", StringType(), True),
        StructField("location", StringType(), True),
        StructField("method", StringType(), True),
        StructField("page", StringType(), True),
        StructField("registration", DoubleType(), True),
        StructField("sessionId", LongType(), True),
        StructField("song", StringType(), True),
        StructField("status", LongType(), True),
        StructField("ts", LongType(), True),
        StructField("userAgent", StringType(), True),
        StructField("userId", StringType(), True),
    ])

In [8]:
john = create_spark_session()

In [9]:
df = john.read.json(log_data, schema=log_data_schema)

In [10]:
print("--- creating the time table ---")
    # create timestamp column from original timestamp column
df = df.withColumn("start_time", F.to_timestamp(F.from_unixtime((col("ts") / 1000) , 'yyyy-MM-dd HH:mm:ss.SSS')).cast("Timestamp"))

--- creating the time table ---


In [11]:
time_table = df.select("start_time").dropDuplicates()

In [12]:
import  pyspark.sql.functions as F

In [13]:
time_table.show(10)

+-------------------+
|         start_time|
+-------------------+
|2018-11-15 07:49:47|
|2018-11-15 19:43:20|
|2018-11-21 02:40:02|
|2018-11-21 10:52:12|
|2018-11-21 13:48:37|
|2018-11-21 19:46:29|
|2018-11-14 04:37:40|
|2018-11-14 12:14:41|
|2018-11-14 16:19:02|
|2018-11-14 23:03:00|
+-------------------+
only showing top 10 rows



In [14]:
time_table = time_table.withColumn("month", F.month("start_time"))
time_table = time_table.withColumn("year", F.year("start_time"))
time_table = time_table.withColumn("day", F.dayofmonth("start_time"))
time_table = time_table.withColumn("hour", F.hour("start_time"))
time_table = time_table.withColumn("week", F.weekofyear("start_time"))
time_table = time_table.withColumn("weekday", F.dayofweek("start_time"))

time_table.show(5)

+-------------------+-----+----+---+----+----+-------+
|         start_time|month|year|day|hour|week|weekday|
+-------------------+-----+----+---+----+----+-------+
|2018-11-15 07:49:47|   11|2018| 15|   7|  46|      5|
|2018-11-15 19:43:20|   11|2018| 15|  19|  46|      5|
|2018-11-21 02:40:02|   11|2018| 21|   2|  47|      4|
|2018-11-21 10:52:12|   11|2018| 21|  10|  47|      4|
|2018-11-21 13:48:37|   11|2018| 21|  13|  47|      4|
+-------------------+-----+----+---+----+----+-------+
only showing top 5 rows

